In [ ]:
import credentials
import settings
import mysql.connector
import pandas as pd
import time
import textblob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import nltk
import re
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# LOAD DATA FROM MYSQL
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=credentials.MYSQLPASSWORD,
    database="twitterdb",
    auth_plugin='mysql_native_password',
    charset = 'utf8'
)

df = pd.read_sql('SELECT id_str, text, created_at, polarity, FROM {}'.format(settings.TABLE_NAME), con=db_connection)
# DATE TIME
df['created_at']=pd.to_datetime(df['created_at'])
for index,tweets in df[df['polarity']== -1].iterrows():
    print(" "+str(tweets[2])+ " "+tweets[1])

In [ ]:
#SENTIMENT ANALYSIS WITH TEXTBLOB - NEGATIVE TWEETS ARE -1, POSITIVE TWEETS ARE +1, AND NEUTRAL TWEETS ARE 0. DONE BY THE PREPROCESSING FOR BETTER DATA STORAGE
from textblob import TextBlob
sentiment=TextBlob(tweet_text).sentiment
polarity=sentiment.polarity
subjectivity=sentiment.subjectivity

In [ ]:
#CLEAN AND TRANSFORM DATA TO ENABLE TIME SERIES
result=df.groupby(
    [pd.Grouper(key='created_at',freq='2s'),'polarity']).count().unstack(fill_value=0).stack().reset_index()
result=result.rename(columns=
                     {"id_str":"Num of '{}' mentions".format("Ethereum"),"created_at":"Time in ET"})
time_series=result["Time in ET"][result['polarity']==0].reset_index(drop=True)

fig=px.line(result,x='Time in ET',
            y="Num of '{}' mentions".format("Ethereum"),color='polarity')
fig.show()

In [ ]:
#TRACKING MOST FREQUENT VOCABULARY USED IN TWEETS

#CLEANING
content = ' '.join(df["text"])
content = re.sub(r"http\S+", "", content)
content = content.replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()


tokenwords=word_tokenize(content)
stop_words=set(stopwords.words('english'))
filtered_sen=[]

for i in tokenwords:
    if i not in stop_words:
        filtered_sen.append(i)
fdist=FreqDist(filtered_sen)
fd=pd.DataFrame(fdist.most_common(20),
                columns=["Word","Frequency"]).drop([0]).reindex()
     

In [8]:
# POTENTIALLY ADDING NEW STOPWORDS TO STOP_WORDS DICTIONARY.
new_stopwords = []
stop_words.extend()